In [ ]:
import os
import matplotlib.pyplot as plt
from typing import Dict
from functools import partial, reduce
from tqdm import tqdm
from datetime import timedelta
import tikzplotlib
import json

# Speed comparison

## Line plots

In [ ]:
def _read_metrics(data_dir: str, subdir: str) -> Dict[str, str]:
    with open(os.path.join(data_dir, subdir, 'metric.log')) as file:
        lines = file.readlines()

    ret = {}
    for line in lines:
        name, value = line.split(':')
        _, field = name.split('.', 1)
        val, _ = value.split('|', 1)

        if field not in ret:
            ret[field] = []

        ret[field].append(float(val))

    return ret

def _read_data(data_dir: str) -> Dict[str, Dict[str, str]]:
    is_dir = lambda x: os.path.isdir(os.path.join(data_dir, x))
    dirs = filter(is_dir, os.listdir(data_dir))
    data = {
        i:_read_metrics(data_dir, i)
            for i in tqdm(dirs)
    }

    return data

def _filter_metric(data, metric: str):
    return {
        index:instance[metric] for index, instance in data.items()
    }

def _prepare_data(metric: str, data_dir: str):
    return _filter_metric(_read_data(data_dir), metric)

In [ ]:
def tikzplotlib_fix_ncols(obj):
    """
    workaround for matplotlib 3.6 renamed legend's _ncol to _ncols, which breaks tikzplotlib
    """
    if hasattr(obj, "_ncols"):
        obj._ncol = obj._ncols
    for child in obj.get_children():
        tikzplotlib_fix_ncols(child)

In [ ]:
def _read_ram(data_dir: str):
    with open(os.path.join(data_dir, 'ram.log')) as file:
        return [ json.loads(i) for i in file.readlines() ]        

In [ ]:
VALUES = {}
RAM = {}

### Native serialization mechanism

In [ ]:
DATA_DIR = './experiments3/Speed_test_normal_mode/'
DATA = _read_data(DATA_DIR)
VALUES['Native'] = DATA
RAM['Native'] = _read_ram(DATA_DIR)

In [ ]:
TIME = 14400
TICKS = 10
METRIC = 'execs_per_sec'
XLABEL = 'Time in hours : minutes : seconds'
YLABEL = 'Test case execution speed per second'
TITLE = 'Native serialization mechanism speed'
OUT = 'normal_speed.pdf'

fig, ax = plt.subplots()
data = _filter_metric(DATA, METRIC)
max_x = max(len(val) for _, val in data.items())

ax.set_ylim([0, 6])
xticks = [ i / TICKS for i in range(TICKS) ]
xlabels = [ timedelta(seconds=int(TIME * i)) for i in xticks ]
ax.set_xticks(xticks, xlabels, rotation=45)
ax.set_ylabel(YLABEL)
ax.set_xlabel(XLABEL)
ax.set_title(TITLE)
fig.tight_layout()

for _, val in data.items():
    xlen = len(val)
    x = [ i / xlen for i in range(xlen) ]
    ax.plot(x, val)

fig.savefig(OUT)

In [ ]:
TIME = 14400
TICKS = 10
METRIC = 'total_crashes'
XLABEL = 'Time in hours : minutes : seconds'
YLABEL = 'Total crashes collected'
TITLE = 'Native serialization mechanism crashes'
OUT = 'normal_crashes.pdf'

fig, ax = plt.subplots()
data = _filter_metric(DATA, METRIC)
max_x = max(len(val) for _, val in data.items())

#ax.set_ylim([0, 6])
xticks = [ i / TICKS for i in range(TICKS) ]
xlabels = [ timedelta(seconds=int(TIME * i)) for i in xticks ]
ax.set_xticks(xticks, xlabels, rotation=45)
ax.set_ylabel(YLABEL)
ax.set_xlabel(XLABEL)
ax.set_title(TITLE)
fig.tight_layout()

for _, val in data.items():
    xlen = len(val)
    x = [ i / xlen for i in range(xlen) ]
    ax.plot(x, val)

fig.savefig(OUT)

### Custom serialization mechanism

In [ ]:
DATA_DIR = './experiments3/Speed_test_fast_mode/'
DATA = _read_data(DATA_DIR)
VALUES['Custom'] = DATA
RAM['Custom'] = _read_ram(DATA_DIR)

In [ ]:
TIME = 14400
TICKS = 10
METRIC = 'execs_per_sec'
XLABEL = 'Time in hours : minutes : seconds'
YLABEL = 'Test case execution speed per second'
TITLE = 'Custom serialization mechanism speed'
OUT = 'fast_speed.pdf'

fig, ax = plt.subplots()
data = _filter_metric(DATA, METRIC)
max_x = max(len(val) for _, val in data.items())

ax.set_ylim([0, 20])
xticks = [ i / TICKS for i in range(TICKS) ]
xlabels = [ timedelta(seconds=int(TIME * i)) for i in xticks ]
ax.set_xticks(xticks, xlabels, rotation=45)
ax.set_ylabel(YLABEL)
ax.set_xlabel(XLABEL)
ax.set_title(TITLE)
fig.tight_layout()

for _, val in data.items():
    xlen = len(val)
    x = [ i / xlen for i in range(xlen) ]
    ax.plot(x, val)

fig.savefig(OUT)

In [ ]:
TIME = 14400
TICKS = 10
METRIC = 'total_crashes'
XLABEL = 'Time in hours : minutes : seconds'
YLABEL = 'Total crashes collected'
TITLE = 'custom serialization mechanism crashes'
OUT = 'custom_crashes.pdf'

fig, ax = plt.subplots()
data = _filter_metric(DATA, METRIC)
max_x = max(len(val) for _, val in data.items())

#ax.set_ylim([0, 6])
xticks = [ i / TICKS for i in range(TICKS) ]
xlabels = [ timedelta(seconds=int(TIME * i)) for i in xticks ]
ax.set_xticks(xticks, xlabels, rotation=45)
ax.set_ylabel(YLABEL)
ax.set_xlabel(XLABEL)
ax.set_title(TITLE)
fig.tight_layout()

for _, val in data.items():
    xlen = len(val)
    x = [ i / xlen for i in range(xlen) ]
    ax.plot(x, val)

fig.savefig(OUT)

### No revert mode, fuzzing from normal world

In [ ]:
DATA_DIR = './experiments/Speed_test_norevert_mode/'
DATA = _read_data(DATA_DIR)
VALUES['No revert'] = DATA
RAM['No revert'] = _read_ram(DATA_DIR)

In [ ]:
TIME = 14400
TICKS = 10
METRIC = 'execs_per_sec'
XLABEL = 'Time in hours : minutes : seconds'
YLABEL = 'Test case execution speed per second'
TITLE = 'Fuzzing without reverting to previous state'
OUT = 'norevert_speed.pdf'

fig, ax = plt.subplots()
data = _filter_metric(DATA, METRIC)
max_x = max(len(val) for _, val in data.items())

ax.set_ylim([0, 100])
xticks = [ i / TICKS for i in range(TICKS) ]
xlabels = [ timedelta(seconds=int(TIME * i)) for i in xticks ]
ax.set_xticks(xticks, xlabels, rotation=45)
ax.set_ylabel(YLABEL)
ax.set_xlabel(XLABEL)
ax.set_title(TITLE)
fig.tight_layout()

for _, val in data.items():
    xlen = len(val)
    x = [ i / xlen for i in range(xlen) ]
    ax.plot(x, val)

fig.savefig(OUT)

In [ ]:
TIME = 14400
TICKS = 10
METRIC = 'total_crashes'
XLABEL = 'Time in hours : minutes : seconds'
YLABEL = 'Total crashes collected'
TITLE = 'No revert crashes'
OUT = 'norevert_crashes.pdf'

fig, ax = plt.subplots()
data = _filter_metric(DATA, METRIC)
max_x = max(len(val) for _, val in data.items())

#ax.set_ylim([0, 6])
xticks = [ i / TICKS for i in range(TICKS) ]
xlabels = [ timedelta(seconds=int(TIME * i)) for i in xticks ]
ax.set_xticks(xticks, xlabels, rotation=45)
ax.set_ylabel(YLABEL)
ax.set_xlabel(XLABEL)
ax.set_title(TITLE)
fig.tight_layout()

for _, val in data.items():
    xlen = len(val)
    x = [ i / xlen for i in range(xlen) ]
    ax.plot(x, val)

fig.savefig(OUT)

### No revert from secure world

## Boxplot

In [ ]:
DATA_DIR = './experiments/Speed_test_tznorevert_mode/'
DATA = _read_data(DATA_DIR)
VALUES['Secure no revert'] = DATA
RAM['Secure no revert'] = _read_ram(DATA_DIR)

In [ ]:
TIME = 14400
TICKS = 10
METRIC = 'execs_per_sec'
XLABEL = 'Time in hours : minutes : seconds'
YLABEL = 'Test case execution speed per second'
TITLE = 'Fuzzing without reverting to previous state'
OUT = 'tznorevert_speed.pdf'

fig, ax = plt.subplots()
data = _filter_metric(DATA, METRIC)
max_x = max(len(val) for _, val in data.items())

ax.set_ylim([0, 100])
xticks = [ i / TICKS for i in range(TICKS) ]
xlabels = [ timedelta(seconds=int(TIME * i)) for i in xticks ]
ax.set_xticks(xticks, xlabels, rotation=45)
ax.set_ylabel(YLABEL)
ax.set_xlabel(XLABEL)
ax.set_title(TITLE)
fig.tight_layout()

for _, val in data.items():
    xlen = len(val)
    x = [ i / xlen for i in range(xlen) ]
    ax.plot(x, val)

fig.savefig(OUT)

In [ ]:
TIME = 14400
TICKS = 10
METRIC = 'total_crashes'
XLABEL = 'Time in hours : minutes : seconds'
YLABEL = 'Total crashes collected'
TITLE = 'Secure no revert crashes'
OUT = 'tznorevert_crashes.pdf'

fig, ax = plt.subplots()
data = _filter_metric(DATA, METRIC)
max_x = max(len(val) for _, val in data.items())

#ax.set_ylim([0, 6])
xticks = [ i / TICKS for i in range(TICKS) ]
xlabels = [ timedelta(seconds=int(TIME * i)) for i in xticks ]
ax.set_xticks(xticks, xlabels, rotation=45)
ax.set_ylabel(YLABEL)
ax.set_xlabel(XLABEL)
ax.set_title(TITLE)
fig.tight_layout()

for _, val in data.items():
    xlen = len(val)
    x = [ i / xlen for i in range(xlen) ]
    ax.plot(x, val)

fig.savefig(OUT)

### Speed

In [ ]:
def _join_values(data):
    return reduce(lambda x, y: x + y, data.values())

In [ ]:
METRIC = 'execs_per_sec'
TITLE = "Comparison of fuzzing speeds"
YLABEL = "Test case execution speed in executions per second"
OUT = 'speed_boxplot.pdf'

fig, ax = plt.subplots()
data = { name:_join_values(_filter_metric(val, METRIC)) for name, val in VALUES.items() }
ax.boxplot(data.values(), labels=data.keys())
ax.set_ylabel(YLABEL)
ax.set_title(TITLE)
ax.set_ylim([0, 20])
fig.tight_layout()

fig.savefig(OUT)

### Crashes

In [ ]:
METRIC = 'total_crashes'
TITLE = "Comparison of total crashes found"
YLABEL = "Total number of crashes registered"
OUT = 'crashes_boxplot.pdf'

fig, ax = plt.subplots()
data = { name:_join_values(_filter_metric(val, METRIC)) for name, val in VALUES.items() }
ax.boxplot(data.values(), labels=data.keys())
ax.set_ylabel(YLABEL)
ax.set_title(TITLE)
fig.tight_layout()

fig.savefig(OUT)

## Ram usage

In [ ]:
TITLE = 'Ram usage by different serialization methods'
XLABEL = 'Time in hours : minutes : seconds'
YLABEL = 'Allocated RAM in GB'
TYPE = 'used'
TIME = 14400
TICKS = 10
OUT = 'ram_line.pdf'

data = { name:[ i[TYPE] for i in val ] for name, val in RAM.items() }
fig, ax = plt.subplots()
xticks = [ i / TICKS for i in range(TICKS) ]
xlabels = [ timedelta(seconds=int(TIME * i)) for i in xticks ]
ax.set_xticks(xticks, xlabels, rotation=45)

for name, val in data.items():
    xlen = len(val)
    x = [ i / xlen for i in range(xlen) ]
    y = [ i / 10 ** 9 for i in val ]
    ax.plot(x, y, label=name)

ax.legend()
ax.set_ylabel(YLABEL)
ax.set_xlabel(XLABEL)
ax.set_title(TITLE)
fig.tight_layout()

fig.savefig(OUT)

In [ ]:
TITLE = 'Ram usage by different serialization methods'
XLABEL = 'Time in hours : minutes : seconds'
YLABEL = 'Allocated RAM in GB'
TYPE = 'used'
TIME = 14400
TICKS = 10
OUT = 'ram_box.pdf'

data = { name:[ i[TYPE] for i in val ] for name, val in RAM.items() }
fig, ax = plt.subplots()

Y = []
labels = []
for name, val in data.items():
    Y.append([ i / 10 ** 9 for i in val ])
    labels.append(name)

ax.boxplot(Y, labels=labels)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

ax.set_ylim([8, 42])
ax.set_ylabel(YLABEL)
ax.set_title(TITLE)
fig.tight_layout()

fig.savefig(OUT)

# Structured fuzzing vs direct

In [ ]:
DATA_DIR = './experiments3/Speed_test_tznorevert_mode_direct/'
DATA = _read_data(DATA_DIR)

### Line plot

In [ ]:
TITLE = 'Structured and direct fuzzing comparison'
XLABEL = 'Time in hours : minutes : seconds'
YLABEL = 'Total crashes'
METRIC = 'total_crashes'
VERSUS = VALUES['Secure no revert']
OUT = 'dsl_direct_line.pdf'

fig, ax = plt.subplots()
data = _filter_metric(DATA, METRIC)
xticks = [ i / TICKS for i in range(TICKS) ]
xlabels = [ timedelta(seconds=int(TIME * i)) for i in xticks ]
ax.set_xticks(xticks, xlabels, rotation=45)

label = 'direct'
for _, val in data.items():
    xlen = len(val)
    x = [ i / xlen for i in range(xlen) ]
    ax.plot(x, val, label=label, color='tab:pink')
    if label is not None:
        label = None

data = _filter_metric(VERSUS, METRIC)
label = 'structured'
for _, val in data.items():
    xlen = len(val)
    x = [ i / xlen for i in range(xlen) ]
    ax.plot(x, val, label=label, color='tab:green')
    if label is not None:
        label = None

    
ax.legend()
ax.set_ylabel(YLABEL)
ax.set_xlabel(XLABEL)
ax.set_title(TITLE)
fig.tight_layout()

fig.savefig(OUT)

### Box plot

In [ ]:
TITLE = 'Structured and direct fuzzing comparison'
YLABEL = 'Total crashes'
METRIC = 'total_crashes'
VERSUS = VALUES['Secure no revert']
OUT = 'dsl_direct_line.pdf'

fig, ax = plt.subplots()
data = {
    "direct":_join_values(_filter_metric(DATA, METRIC)),
    "dsl":_join_values(_filter_metric(VERSUS, METRIC))
}

ax.boxplot(data.values(), labels=data.keys())

ax.set_title(TITLE)
ax.set_ylabel(YLABEL)
fig.tight_layout()

fig.savefig(OUT)

# Seeding corpus with unit tests